In [4]:
import chess
from stockfish import Stockfish
import re
import requests
from ollama import chat
from ollama import ChatResponse
import json
import pandas as pd


In [5]:
stockfish = Stockfish()

In [6]:
# URL to scrape the text from.
url = "https://wtharvey.com/m8n2.txt"

# Download the text from the website.
response = requests.get(url)
if response.status_code == 200:
    text = response.text
else:
    print(f"Failed to retrieve content. Status code: {response.status_code}")
    text = ""

# Regular expression pattern for matching FEN strings.
fen_pattern = re.compile(
    r'([prnbqkPRNBQK\d/]+\s+[wb]\s+(?:K?Q?k?q?|-)\s+(?:[a-h][36]|-)\s+\d+\s+\d+)'
)

# Find all FEN notations in the text.
fen_notations = fen_pattern.findall(text)

# Print the extracted FEN notations.
print("Extracted FEN Notations:")
# for fen in fen_notations:
#     print(fen)

print(fen_notations[:3])

Extracted FEN Notations:
['r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 1 0', '1rb4r/pkPp3p/1b1P3n/1Q6/N3Pp2/8/P1P3PP/7K w - - 1 0', '4kb1r/p2n1ppp/4q3/4p1B1/4P3/1Q6/PPP2PPP/2KR4 w k - 1 0']


In [14]:
fen_notations_ten = fen_notations[:10]

len(fen_notations_ten)

print(fen_notations_ten[0])

r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 1 0


In [8]:
#Create function for getting and parsing llama response here
#Function should return only move in SAN notation
def get_qwen_move(fen):
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn.

    This is the current state of the game. Use this to work out where the pieces are on the board:

    FEN: {fen}

    Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.

    {{
    "san": "The move in SAN format",
    "reason": "Why this is a good move"
    }}
    """
            }
        ])
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    return qwen_move

In [9]:
results = []

In [15]:
for fen in fen_notations_ten:
    if not stockfish.is_fen_valid(fen): exit
    
    print(f"Processing position: {fen}")

    board_stockfish = chess.Board(fen=fen)
    board_qwen = chess.Board(fen=fen)

    stockfish.set_fen_position(fen_position=fen)

    best_move = stockfish.get_best_move()

    board_stockfish.push(chess.Move.from_uci(best_move))
    # best_eval_info = stockfish.get_evaluation()
    best_eval = stockfish.get_evaluation()["value"]  # evaluation in centipawns (or mate score)
    # print(f"Stockfish best move: {best_move} with evaluation: {best_eval}")




    qwen_move = get_qwen_move(fen)

    print(f"QWEN MOVE: {qwen_move}\nQWEN MOVE TYPE: {type(qwen_move)}")

    try:
        chess.Move.from_uci(board_qwen.parse_san(qwen_move))
    except Exception as e:
        print(f"Invalid move geenrated by Qwen. The move is: {qwen_move}")
        continue

    stockfish.set_fen_position(fen_position=board_qwen.fen())
    provided_eval = stockfish.get_evaluation()["value"]

    score_difference = best_eval - provided_eval

    results.append({
        "Best Move Score": best_eval,
        "Move made by Qwen:32B Score": provided_eval,
        "Score Difference": score_difference
    })


Processing position: r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 1 0
QWEN MOVE: Nh3
QWEN MOVE TYPE: <class 'str'>
Invalid move geenrated by Qwen. The move is: Nh3
Processing position: 1rb4r/pkPp3p/1b1P3n/1Q6/N3Pp2/8/P1P3PP/7K w - - 1 0
QWEN MOVE: Nh3
QWEN MOVE TYPE: <class 'str'>
Invalid move geenrated by Qwen. The move is: Nh3
Processing position: 4kb1r/p2n1ppp/4q3/4p1B1/4P3/1Q6/PPP2PPP/2KR4 w k - 1 0
QWEN MOVE: Nf3
QWEN MOVE TYPE: <class 'str'>
Invalid move geenrated by Qwen. The move is: Nf3
Processing position: r1b2k1r/ppp1bppp/8/1B1Q4/5q2/2P5/PPP2PPP/R3R1K1 w - - 1 0
QWEN MOVE: Nf3
QWEN MOVE TYPE: <class 'str'>
Invalid move geenrated by Qwen. The move is: Nf3
Processing position: 5rkr/pp2Rp2/1b1p1Pb1/3P2Q1/2n3P1/2p5/P4P2/4R1K1 w - - 1 0
QWEN MOVE: Nh3
QWEN MOVE TYPE: <class 'str'>
Invalid move geenrated by Qwen. The move is: Nh3
Processing position: 1r1kr3/Nbppn1pp/1b6/8/6Q1/3B1P2/Pq3P1P/3RR1K1 w - - 1 0
QWEN MOVE: Nf3
QWEN MOVE TYPE: <class 'str'>
Invalid move g

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")

In [ ]:
You are playing white and it is your turn.

This is the current state of the game use this to work out where the pieces are on the board:

FEN: r2qkb1r/pp2nppp/3p4/2pNN1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 1 0

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.

{
  "san": "The move in SAN format",
  "reason": "Why this is a good move"
}